In [5]:
from helpers import *
from lstmHelpers import *

from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import time
import itertools
import IPython.display as ipd

filedir = "../"

In [6]:
def extract_class_info(matrix):
    data_dict = {}
    match_dict = {}
    
    for i in range(len(audio_matrix)):
        filename = audio_matrix[i][2]
        if filename not in data_dict:
            yt,sr = librosa.load(filename)
            y, idx = librosa.effects.trim(yt, top_db=50)
            data_dict[filename] = y
            match_dict[filename] = audio_matrix[i][1]
    return data_dict, match_dict

audio_matrix = load_stream(name="trainingInfo.txt")
data_dict, match_dict = extract_class_info(audio_matrix)
model = load_model('LSTMModel.h5')

In [9]:
def create_sample_stream(data_dict, match_dict, total_length=30):
    
    total_length *= 22050
    
    composite_signal_list = []
    composite_matches_list = []
    
    cur_length = 0
    num_files_used = 0
    while cur_length < total_length:
        filename = random.choice(list(data_dict.keys()))
        y = data_dict.get(filename)
        noisey_y = apply_noise(y, random.randint(0, 600) / 10.0)
        faded_y = apply_ramp(noisey_y, random.randint(0, int(0.25*22050)), random.randint(0, int(0.25*22050)))
        composite_signal_list.append(faded_y)
        composite_matches_list.append(np.full(faded_y.shape, match_dict.get(filename)))
        cur_length += faded_y.shape[0]
        num_files_used += 1
    composite_signal = np.array(composite_signal_list)
    composite_signal = np.concatenate(composite_signal).ravel()
    composite_matches = np.array(composite_matches_list)
    composite_matches = np.concatenate(composite_matches).ravel()
    return composite_signal, composite_matches

test_signal, test_matches = create_sample_stream(data_dict, match_dict, total_length=30)
ipd.Audio(test_signal, rate=22050)

In [10]:
batch_length = 20
n_mels = 128

recent_frames = []
cur_frame_count = 0

num_frames_to_count = 2048*5
true = 0
false = 0

for frame in np.nditer(test_signal):
    recent_frames.append(frame)
    cur_frame_count += 1
    if len(recent_frames) == num_frames_to_count:
        recent_signal = np.asarray(recent_frames)
        recent_signal = np.pad(recent_signal, (0, 2048*5), 'constant', constant_values=(0.0,0.0))
        spec = get_spectrogram(recent_signal, 22050, n_mels=n_mels, display=False)
        transposed = spec.T
        scaler = MinMaxScaler(feature_range=(0, 1))
        transposed = scaler.fit_transform(transposed)
        comp_cols = []
        for i in range(0,batch_length):
            comp_cols.append(transposed[i])
        comp_cols = np.array([comp_cols])
        test_output = test_matches[cur_frame_count - num_frames_to_count]
        prediction = model.predict_classes(comp_cols)
        print(str(prediction[0]) + " : " + str(test_output) + "    ", end="")
        if int(test_output) == int(prediction[0]):
            true += 1
        else:
            false += 1
        recent_frames = []

print(true / (false+true))

1 : 1    3 : 1    5 : 5    3 : 5    1 : 5    1 : 0    2 : 0    1 : 1    1 : 1    3 : 1    1 : 1    3 : 0    5 : 0    0 : 0    1 : 4    1 : 4    1 : 3    3 : 3    1 : 3    1 : 3    1 : 3    4 : 4    1 : 4    1 : 1    1 : 1    3 : 1    1 : 1    2 : 6    5 : 6    1 : 0    0 : 0    2 : 6    5 : 6    1 : 6    5 : 6    1 : 0    1 : 0    1 : 2    1 : 2    1 : 2    3 : 2    1 : 2    3 : 3    1 : 3    1 : 4    4 : 4    1 : 4    1 : 4    4 : 4    4 : 4    1 : 4    1 : 4    1 : 4    5 : 7    7 : 7    1 : 7    1 : 7    1 : 8    1 : 8    1 : 5    1 : 5    1 : 5    1 : 5    1 : 3    1 : 3    1 : 0    0.25757575757575757
